## 1. Setup - Colab Kontrolü ve Proje Kurulumu

In [ ]:
import sys
import os

# Colab kontrolü
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("✅ Google Colab ortamı tespit edildi")
    
    # GPU kontrolü
    import torch
    if torch.cuda.is_available():
        print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
        print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    else:
        print("⚠️  GPU bulunamadı! Runtime > Change runtime type > GPU seçin")
    
    # GitHub'dan projeyi klonla
    print("\n📥 Proje indiriliyor...")
    !git clone https://github.com/Aliekinozcetin/Mitre_Attack_TTP_Mapping.git
    
    # Proje dizinine geç
    os.chdir('Mitre_Attack_TTP_Mapping')
    print(f"✅ Çalışma dizini: {os.getcwd()}")
    
    # Gerekli paketleri yükle
    print("\n📦 Paketler yükleniyor...")
    !pip install -q -r requirements.txt
    print("✅ Tüm paketler yüklendi")
else:
    print("ℹ️  Yerel ortamda çalışıyorsunuz")

## 2. Import Modüller

In [ ]:
import torch
import json
from datetime import datetime

from src.data_loader import prepare_data
from src.model import load_model
from src.train import train_model
from src.evaluate import evaluate_model

print("✅ Modüller yüklendi")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

## 3. Konfigürasyon

In [ ]:
# Training parametreleri
CONFIG = {
    'model_name': 'bert-base-uncased',  # veya 'jackaduma/SecBERT', 'distilbert-base-uncased'
    'max_length': 512,
    'batch_size': 16,  # GPU varsa 32'ye çıkarabilirsin
    'learning_rate': 2e-5,
    'num_epochs': 3,
    'warmup_steps': 500,
    'threshold': 0.5,
    'device': 'cuda' if torch.cuda.is_available() else 'cpu',
    'output_dir': './outputs'
}

# Konfigürasyonu yazdır
print("\n" + "="*70)
print("TRAINING CONFIGURATION")
print("="*70)
for key, value in CONFIG.items():
    print(f"  {key:20s}: {value}")
print("="*70)

## 4. Veri Yükleme ve Hazırlama

In [ ]:
print("\n" + "="*70)
print("STEP 1: DATA PREPARATION")
print("="*70 + "\n")

data = prepare_data(
    model_name=CONFIG['model_name'],
    max_length=CONFIG['max_length']
)

train_dataset = data['train_dataset']
test_dataset = data['test_dataset']
label_list = data['label_list']
num_labels = data['num_labels']

print(f"\n✅ Veri hazırlama tamamlandı!")
print(f"   Train samples: {len(train_dataset)}")
print(f"   Test samples: {len(test_dataset)}")
print(f"   Number of labels: {num_labels}")

## 5. Model Yükleme

In [ ]:
print("\n" + "="*70)
print("STEP 2: MODEL INITIALIZATION")
print("="*70 + "\n")

model = load_model(
    model_name=CONFIG['model_name'],
    num_labels=num_labels,
    device=CONFIG['device']
)

print(f"\n✅ Model yüklendi ve {CONFIG['device']} cihazına taşındı!")

## 6. Model Eğitimi

In [ ]:
print("\n" + "="*70)
print("STEP 3: MODEL TRAINING")
print("="*70 + "\n")

# Output dizini oluştur
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
run_name = f"{CONFIG['model_name'].replace('/', '_')}_{timestamp}"
output_dir = os.path.join(CONFIG['output_dir'], run_name)
os.makedirs(output_dir, exist_ok=True)

# Label listesini kaydet
label_file = os.path.join(output_dir, "labels.json")
with open(label_file, 'w') as f:
    json.dump(label_list, f, indent=2)

# Eğitimi başlat
history = train_model(
    model=model,
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    output_dir=output_dir,
    batch_size=CONFIG['batch_size'],
    learning_rate=CONFIG['learning_rate'],
    num_epochs=CONFIG['num_epochs'],
    warmup_steps=CONFIG['warmup_steps'],
    device=CONFIG['device']
)

# Training geçmişini kaydet
history_file = os.path.join(output_dir, "training_history.json")
with open(history_file, 'w') as f:
    json.dump(history, f, indent=2)

print(f"\n✅ Eğitim tamamlandı!")
print(f"   Final loss: {history['train_loss'][-1]:.4f}")

## 7. Model Değerlendirme

In [ ]:
print("\n" + "="*70)
print("STEP 4: MODEL EVALUATION")
print("="*70 + "\n")

metrics = evaluate_model(
    model=model,
    test_dataset=test_dataset,
    batch_size=CONFIG['batch_size'],
    device=CONFIG['device'],
    threshold=CONFIG['threshold'],
    label_list=label_list
)

# Metrikleri kaydet
metrics_to_save = {k: float(v) if isinstance(v, (float, int)) else v 
                   for k, v in metrics.items() 
                   if k not in ['predictions', 'labels']}
metrics_file = os.path.join(output_dir, "evaluation_metrics.json")
with open(metrics_file, 'w') as f:
    json.dump(metrics_to_save, f, indent=2)

print(f"\n✅ Değerlendirme tamamlandı!")
print(f"\nKey Metrics:")
print(f"  Micro F1:    {metrics['micro_f1']:.4f}")
print(f"  Macro F1:    {metrics['macro_f1']:.4f}")
print(f"  Samples F1:  {metrics['samples_f1']:.4f}")

## 8. Sonuçları Kaydet ve İndir

In [ ]:
# Özet dosyası oluştur
summary = {
    'model': CONFIG['model_name'],
    'timestamp': timestamp,
    'configuration': CONFIG,
    'data': {
        'num_labels': num_labels,
        'train_samples': len(train_dataset),
        'test_samples': len(test_dataset)
    },
    'training': {
        'final_loss': history['train_loss'][-1]
    },
    'evaluation': metrics_to_save
}

summary_file = os.path.join(output_dir, "summary.json")
with open(summary_file, 'w') as f:
    json.dump(summary, f, indent=2)

print("\n" + "="*70)
print("PIPELINE COMPLETE!")
print("="*70)
print(f"\nSonuçlar kaydedildi: {output_dir}")
print("\nDosyalar:")
print(f"  - labels.json")
print(f"  - training_history.json")
print(f"  - evaluation_metrics.json")
print(f"  - summary.json")
print(f"  - final_model.pt")
print(f"  - checkpoint_epoch_*.pt")
print("="*70)

## 9. Colab'da Sonuçları İndir

In [ ]:
if IN_COLAB:
    import shutil
    from google.colab import files
    
    # Sonuçları ZIP'le
    zip_name = f"{run_name}.zip"
    shutil.make_archive(run_name, 'zip', output_dir)
    
    print(f"\n📦 Sonuçlar sıkıştırılıyor: {zip_name}")
    print(f"   Boyut: {os.path.getsize(zip_name) / (1024*1024):.2f} MB")
    
    # İndir
    print("\n⬇️  İndirme başlatılıyor...")
    files.download(zip_name)
    print("✅ İndirme tamamlandı!")
else:
    print("ℹ️  Yerel ortamdasınız, sonuçlar zaten bilgisayarınızda.")

## 10. (Opsiyonel) Farklı Modelleri Dene

In [ ]:
# SecBERT modelini denemek için bu hücreyi çalıştır
# CONFIG['model_name'] = 'jackaduma/SecBERT'

# DistilBERT modelini denemek için bu hücreyi çalıştır  
# CONFIG['model_name'] = 'distilbert-base-uncased'
# CONFIG['batch_size'] = 32  # DistilBERT daha küçük, batch size artırılabilir

# Sonra yukarıdaki hücreleri tekrar çalıştır